 The code in this notebook, **when run in order**, will:

#### (1) Produce a dataframe of tweets for a certain keyword (columns right now are, location and tweet text; can modify the search criteria as necessary (location lists, hashtags, etc.), saved as a csv

#### (2) Delete the duplicates/very similar spam tweets but leave one copy behind. 

#### (3) Clean the remaining tweets and saves as a csv

By the end of the notebook, you'll have a clean twitter dataframe that's ready for analysis 

In [75]:
import os
import tweepy
import pandas as pd
import time
import itertools
import numpy as np
from itertools import chain


In [76]:
consumer_key= '3emnsuDpqLBXlxD5UPUxVnxTt'
consumer_secret= 'ZPZBilbjbca7pYAmMv05hlNKsxLb4CJd6A5kNa2JNIs62BjF7m'
access_token= '1406213154-M77vYKXz7wIAUzzJkqLVzfDH0D6K11xMJkzGHzJ'
access_token_secret= 'iLcP4BuW4jWRYIHEiIZwsD5jgYlMY1aTmtSFwPQqbaQrX'

In [77]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

## Specify tweet search/filtering criteria here:

In [78]:
search_words = "(mask+free) OR (#maskfree) OR (maskfree) OR (mask-free)"
date_since = "2021-03-10"
new_search = search_words + " -filter:retweets"


## Set location labels and corresponding location coordinates and radii here:

In [79]:
## ** In the order of the counties listed in the google doc, and put in the Facebook chat

locs_labels = ['Texas test',
'New York County NY',
 'County of San Francisco CA',
 'Hudson County NJ',
 'Suffolk County MA',
 'Philadelphia County PA',
 'Washington DC',
 'Alexandria VA',
 'Baltimore MD',
 'Cook County IL',
 'St. Louis County MO',
 'Milwaukee County WI',
 'Denver County CO',
 'Orange County CA',
 'Pinellas County FL',
 'Ramsey County MN',
 'Wayne County, MI',
 'Cuyahoga County OH',
 'Dallas County TX',
 'DeKalb County GA',
 'Salt Lake County UT',
 'Marion County IN',
 'Jefferson County KY',
 'Johnson County KS',
 'King County WA']


locs = ['29.88548725,-96.27736949,100mi',
    '40.7741618671,-73.9697971209,100mi',
    '37.7597150131,-122.693975573,100mi',
    '40.7309049975,-74.0759554422,100mi',
    '42.3486893925,-70.9856096697,100mi',
    '40.0076360466,-75.1339446794,100mi', 
    '38.9047737146,-77.0163026702,100mi',
    '38.8184535854,-77.0862060584,100mi',
    '39.443072852,-76.6163229502,100mi',
    '41.8954290221,-87.6461407931,100mi',
    '38.6406885662,-90.4433975603,100mi',
    '43.0154568883,-87.5805486607,100mi',
    '39.7621646287,-104.875801423,100mi',
    '33.6769134328,-117.776142822,100mi',
    '27.9026762041,-82.7395539464,100mi',
    '45.0170541432,-93.099606274,100mi',
    '42.2847454761,-83.2611328686,100mi',
    '41.6350590585,-81.7001959208,100mi',
    '32.7666300195,-96.7778770663,100mi',
    '33.7715468713,-84.226432416,100mi',
    '40.6673139262,-111.923601307,100mi',
    '39.7817096805,-86.1384716709,100mi',
    '38.1871910002,-85.6591577058,100mi',
    '38.8837685125,-94.8222610425,100mi',
    '47.4909231926,-121.83595173,100mi']

## Just zip locs_labels with locs and you'd have the coordinates mapped to their names

mapped_locations = list(zip(locs_labels, locs))

In [80]:
mapped_locations

[('Texas test', '29.88548725,-96.27736949,100mi'),
 ('New York County NY', '40.7741618671,-73.9697971209,100mi'),
 ('County of San Francisco CA', '37.7597150131,-122.693975573,100mi'),
 ('Hudson County NJ', '40.7309049975,-74.0759554422,100mi'),
 ('Suffolk County MA', '42.3486893925,-70.9856096697,100mi'),
 ('Philadelphia County PA', '40.0076360466,-75.1339446794,100mi'),
 ('Washington DC', '38.9047737146,-77.0163026702,100mi'),
 ('Alexandria VA', '38.8184535854,-77.0862060584,100mi'),
 ('Baltimore MD', '39.443072852,-76.6163229502,100mi'),
 ('Cook County IL', '41.8954290221,-87.6461407931,100mi'),
 ('St. Louis County MO', '38.6406885662,-90.4433975603,100mi'),
 ('Milwaukee County WI', '43.0154568883,-87.5805486607,100mi'),
 ('Denver County CO', '39.7621646287,-104.875801423,100mi'),
 ('Orange County CA', '33.6769134328,-117.776142822,100mi'),
 ('Pinellas County FL', '27.9026762041,-82.7395539464,100mi'),
 ('Ramsey County MN', '45.0170541432,-93.099606274,100mi'),
 ('Wayne County, MI',

In [81]:
len(locs) # ensure that locs and loc_labels lens match; should have through zip funct anyway

25

In [82]:
len(locs_labels)

25

In [83]:
'''
# Nassau county, NY
tweet_lst=[]
geoc="40.6546,73.5594,50mi"
txgeoc = '29.88548725,-96.27736949,100mi'
migeoc = '25.6112362,-80.55170587,50mi'
for tweet in tweepy.Cursor(api.search,q='#maskfree', lang="en", since=date_since, geocode=txgeoc, tweet_mode='extended').items(1000):
    print(tweet)
    if(tweet.coordinates != None):
        print('we got here')
        tweet_lst.append([tweet.id,tweet.coordinates[0],
                          tweet.coordinates[1],
                          tweet.user.screen_name,
                          tweet.user.name, tweet.text])
    else:
        print("it was none")
tweet_df = pd.DataFrame(tweet_lst, columns=['id', 'lat','long','username', 'name', 'tweet'])
'''

'\n# Nassau county, NY\ntweet_lst=[]\ngeoc="40.6546,73.5594,50mi"\ntxgeoc = \'29.88548725,-96.27736949,100mi\'\nmigeoc = \'25.6112362,-80.55170587,50mi\'\nfor tweet in tweepy.Cursor(api.search,q=\'#maskfree\', lang="en", since=date_since, geocode=txgeoc, tweet_mode=\'extended\').items(1000):\n    print(tweet)\n    if(tweet.coordinates != None):\n        print(\'we got here\')\n        tweet_lst.append([tweet.id,tweet.coordinates[0],\n                          tweet.coordinates[1],\n                          tweet.user.screen_name,\n                          tweet.user.name, tweet.text])\n    else:\n        print("it was none")\ntweet_df = pd.DataFrame(tweet_lst, columns=[\'id\', \'lat\',\'long\',\'username\', \'name\', \'tweet\'])\n'

In [84]:
#tweet_df

In [85]:
'''
### TEXAS TEST ####


tweet_lst=[]
geoc="40.6546,73.5594,50mi"
txgeoc = '29.88548725,-96.27736949,100mi'
migeoc = '25.6112362,-80.55170587,50mi'

    

# Creation of query method using parameters
try:
    tweets = tweepy.Cursor(api.search,q='mask free', lang="en", since=date_since, geocode=txgeoc, tweet_mode='extended').items(5) 
# Pulling information from tweets iterable object
    tweets_list = [[tweet.created_at, tweet.id, tweet.full_text] for tweet in tweets]
    tweets_sort = tweets_list.sort()
    no_dupes = list(k for k,_ in itertools.groupby(tweets_sort))
    print(tweets_list)
 # Creation of dataframe from tweets list
 # Add or remove columns as you remove tweet information
    tweets_d = pd.DataFrame(no_dupes)

except BaseException as e:
    print('failed on_status,',str(e))

tweets_f = pd.DataFrame(no_dupes)
'''

'\n### TEXAS TEST ####\n\n\ntweet_lst=[]\ngeoc="40.6546,73.5594,50mi"\ntxgeoc = \'29.88548725,-96.27736949,100mi\'\nmigeoc = \'25.6112362,-80.55170587,50mi\'\n\n    \n\n# Creation of query method using parameters\ntry:\n    tweets = tweepy.Cursor(api.search,q=\'mask free\', lang="en", since=date_since, geocode=txgeoc, tweet_mode=\'extended\').items(5) \n# Pulling information from tweets iterable object\n    tweets_list = [[tweet.created_at, tweet.id, tweet.full_text] for tweet in tweets]\n    tweets_sort = tweets_list.sort()\n    no_dupes = list(k for k,_ in itertools.groupby(tweets_sort))\n    print(tweets_list)\n # Creation of dataframe from tweets list\n # Add or remove columns as you remove tweet information\n    tweets_d = pd.DataFrame(no_dupes)\n\nexcept BaseException as e:\n    print(\'failed on_status,\',str(e))\n\ntweets_f = pd.DataFrame(no_dupes)\n'

## The code that does the actual scraping by location:

In [86]:
# SCRAPING TWEETS FOR ALL LOCATIONS # 

tweets_list = []
for loc in mapped_locations:
    try:
        tweets = tweepy.Cursor(api.search,q=new_search, lang="en", since=date_since, geocode=loc[1], tweet_mode='extended').items(5) 
        tweets_list.append((loc[0], [[tweet.full_text] for tweet in tweets]))
        print(tweets_list)
        tweets_df = pd.DataFrame(tweets_list)

    except BaseException as e:
        print('failed on_status,',str(e))
        time.sleep(3)
    

[('Texas test', [['@tigerfan1975lsu @AmericanAir Thanks @AmericanAir ! Anyway we can get a badge or something to make sure everybody knows we ride with Kerry mask free all flight long? Please give us an idea of what it may look like so we know it’s authentic. Maybe add a barcode for scanning, just a thought.'], ["@Heart10Looks3 @JosephChaplik I'm free to not wear a mask today. I exercise that right every day and everywhere because masks are dumb and don't do anything to reduce the spread of ANY respiratory virus - including the one you're so terrified of. Go cry elsewhere."], ['@johncardillo @AmericanAir @JohnKerry Don’t forget the fine of up to $35k, the @FAANews would have fined me, but @JohnKerry gets a free pass. Either lift the mask requirement or fine his ass'], ['When dining in at restaurants, do y’all take your mask(s) off once you’re seated at the table or after your food and drinks have arrived? Judge free zone.'], ['💜TODAY ONLY🥳 Get a free Purple Mask Packette on all orders!

In [87]:
tweets_df

,0,1
0,Texas test,[[@tigerfan1975lsu @AmericanAir Thanks @Americ...
1,New York County NY,[[@AlwayzSusan @jackflash630 C'mon down to Flo...
2,County of San Francisco CA,"[[If you live in supportive housing, you are p..."
3,Hudson County NJ,[[@AlwayzSusan @jackflash630 C'mon down to Flo...
4,Suffolk County MA,[[Bear Mask https://t.co/xKH1CEVCkX Etsy pre...
5,Philadelphia County PA,[[@AlwayzSusan @jackflash630 C'mon down to Flo...
6,Washington DC,[[You know @CrownRoyal a Master Sgt. in the US...
7,Alexandria VA,[[You know @CrownRoyal a Master Sgt. in the US...
8,Baltimore MD,[[You know @CrownRoyal a Master Sgt. in the US...
9,Cook County IL,[[@weewoowagoneer People are the worst. Today ...


In [88]:
tweets_df.to_csv('tweet3_dataset.csv') # saves the dataframe


################################################################################################

# Tweet df loading and cleaning

In [89]:
import os
import tweepy
import pandas as pd
import time
import itertools
import numpy as np
from itertools import chain
import regex as re
import unicodedata
import nltk

## Loads tweet dataframe, ensures datatypes are appropriate, flattens tweets into one list for ease, labels columns

In [90]:
tweets_df = pd.read_csv('tweet3_dataset.csv', converters={'1': eval}, index_col=0)
tweets_df['tweets_list'] = tweets_df['1'].apply(lambda x : np.array(x).flatten())
tweets_df.columns = ['county', 'original_tweet', 'tweets_list']
tweets_df

,county,original_tweet,tweets_list
0,Texas test,[[@tigerfan1975lsu @AmericanAir Thanks @Americ...,[@tigerfan1975lsu @AmericanAir Thanks @America...
1,New York County NY,[[@AlwayzSusan @jackflash630 C'mon down to Flo...,[@AlwayzSusan @jackflash630 C'mon down to Flor...
2,County of San Francisco CA,"[[If you live in supportive housing, you are p...","[If you live in supportive housing, you are pr..."
3,Hudson County NJ,[[@AlwayzSusan @jackflash630 C'mon down to Flo...,[@AlwayzSusan @jackflash630 C'mon down to Flor...
4,Suffolk County MA,[[Bear Mask https://t.co/xKH1CEVCkX Etsy pre...,[Bear Mask https://t.co/xKH1CEVCkX Etsy pret...
5,Philadelphia County PA,[[@AlwayzSusan @jackflash630 C'mon down to Flo...,[@AlwayzSusan @jackflash630 C'mon down to Flor...
6,Washington DC,[[You know @CrownRoyal a Master Sgt. in the US...,[You know @CrownRoyal a Master Sgt. in the US ...
7,Alexandria VA,[[You know @CrownRoyal a Master Sgt. in the US...,[You know @CrownRoyal a Master Sgt. in the US ...
8,Baltimore MD,[[You know @CrownRoyal a Master Sgt. in the US...,[You know @CrownRoyal a Master Sgt. in the US ...
9,Cook County IL,[[@weewoowagoneer People are the worst. Today ...,[@weewoowagoneer People are the worst. Today a...


In [91]:
# texas  --- 226 total tweets
#for index, tweet in enumerate(list(tweets_df['tweets_list'].iloc[0])):
#    print(index, tweet)
    

## Now, expands dataset so there is one row per tweet (before, all tweets for a county were in one row)


In [92]:
t_explode = tweets_df.explode('original_tweet')
t_explode

,county,original_tweet,tweets_list
0,Texas test,[@tigerfan1975lsu @AmericanAir Thanks @America...,[@tigerfan1975lsu @AmericanAir Thanks @America...
0,Texas test,[@Heart10Looks3 @JosephChaplik I'm free to not...,[@tigerfan1975lsu @AmericanAir Thanks @America...
0,Texas test,[@johncardillo @AmericanAir @JohnKerry Don’t f...,[@tigerfan1975lsu @AmericanAir Thanks @America...
0,Texas test,"[When dining in at restaurants, do y’all take ...",[@tigerfan1975lsu @AmericanAir Thanks @America...
0,Texas test,[💜TODAY ONLY🥳 Get a free Purple Mask Packette ...,[@tigerfan1975lsu @AmericanAir Thanks @America...
...,...,...,...
24,King County WA,[✨New Event!✨\nFree Drive-Thru Food Event!\nCo...,[✨New Event!✨\nFree Drive-Thru Food Event!\nCo...
24,King County WA,[@hzairs @itsjustkendyll you must be very naiv...,[✨New Event!✨\nFree Drive-Thru Food Event!\nCo...
24,King County WA,[✨New Event!✨\nFree Drive-Thru Food Event!\nCo...,[✨New Event!✨\nFree Drive-Thru Food Event!\nCo...
24,King County WA,[Government announce new safety mearsures for ...,[✨New Event!✨\nFree Drive-Thru Food Event!\nCo...


In [93]:
# tex = t_explode.loc[t_explode['county'] == 'Texas test']

# print(len(tex)) # confirm that this is 226; it is

In [94]:
t_explode['original_tweet'].iloc[56]

['@patriottakes @mama_c2 @mtgreenee @Holly_4Congress The same hood rat that always wears the "Free Speech" mask! Oh, that\'s classic. 😂']

***********
# See note:

I tried getting rid of duplicates a few ways; doing pairwise levenshtein distance comparisons between all the tweets took way too long, and creating a mask to then drop duplicate tweets was also complicated. I noticed that most very-similar-basically-duplicate tweets had the same text, but different handles and urls. So I took a slice of part of the text that was very likely to be shared between the similar tweets, created a new column in the dataset where each tweet-row got a corresponding tweet-slice, and then I deleted the rows where there were duplicate tweet-slices. As you can see, this resulted in a *lot* of data loss (3000+ tweets down to 850; see twitter_drop a few cells down), because many of them were duplicate/spam tweets.
***********

In [95]:
# converting to string data type 
t_explode["tweet_str"]= t_explode["original_tweet"].astype(str) 
  
# slicing till 2nd last element 
t_explode["tweet_slice"] = t_explode["tweet_str"].str.slice(50,1000) 
  
# display 
t_explode

,county,original_tweet,tweets_list,tweet_str,tweet_slice
0,Texas test,[@tigerfan1975lsu @AmericanAir Thanks @America...,[@tigerfan1975lsu @AmericanAir Thanks @America...,['@tigerfan1975lsu @AmericanAir Thanks @Americ...,r ! Anyway we can get a badge or something to ...
0,Texas test,[@Heart10Looks3 @JosephChaplik I'm free to not...,[@tigerfan1975lsu @AmericanAir Thanks @America...,"[""@Heart10Looks3 @JosephChaplik I'm free to no...",ar a mask today. I exercise that right every d...
0,Texas test,[@johncardillo @AmericanAir @JohnKerry Don’t f...,[@tigerfan1975lsu @AmericanAir Thanks @America...,['@johncardillo @AmericanAir @JohnKerry Don’t ...,"et the fine of up to $35k, the @FAANews would ..."
0,Texas test,"[When dining in at restaurants, do y’all take ...",[@tigerfan1975lsu @AmericanAir Thanks @America...,"['When dining in at restaurants, do y’all take...",r mask(s) off once you’re seated at the table ...
0,Texas test,[💜TODAY ONLY🥳 Get a free Purple Mask Packette ...,[@tigerfan1975lsu @AmericanAir Thanks @America...,['💜TODAY ONLY🥳 Get a free Purple Mask Packette...,all orders! Use promo code: HBDPURPLE! https:/...
...,...,...,...,...,...
24,King County WA,[✨New Event!✨\nFree Drive-Thru Food Event!\nCo...,[✨New Event!✨\nFree Drive-Thru Food Event!\nCo...,['✨New Event!✨\nFree Drive-Thru Food Event!\nC...,to Marymoor Park at 6046 W Lk Sammamish Pkwy N...
24,King County WA,[@hzairs @itsjustkendyll you must be very naiv...,[✨New Event!✨\nFree Drive-Thru Food Event!\nCo...,['@hzairs @itsjustkendyll you must be very nai...,o not see how many ppl are voicing out about i...
24,King County WA,[✨New Event!✨\nFree Drive-Thru Food Event!\nCo...,[✨New Event!✨\nFree Drive-Thru Food Event!\nCo...,['✨New Event!✨\nFree Drive-Thru Food Event!\nC...,to Park Ridge Community Church at 3805 Maltby ...
24,King County WA,[Government announce new safety mearsures for ...,[✨New Event!✨\nFree Drive-Thru Food Event!\nCo...,['Government announce new safety mearsures for...,en and girls... a free Churchill face mask\n\n...


### Can look through slice list to ensure that the slices look right

In [96]:
slice_list = t_explode['tweet_slice'].tolist()


for i, v in enumerate(slice_list):
    print(i,v)
#slice_list[184]

0 r ! Anyway we can get a badge or something to make sure everybody knows we ride with Kerry mask free all flight long? Please give us an idea of what it may look like so we know it’s authentic. Maybe add a barcode for scanning, just a thought.']
1 ar a mask today. I exercise that right every day and everywhere because masks are dumb and don't do anything to reduce the spread of ANY respiratory virus - including the one you're so terrified of. Go cry elsewhere."]
2 et the fine of up to $35k, the @FAANews would have fined me, but @JohnKerry gets a free pass. Either lift the mask requirement or fine his ass']
3 r mask(s) off once you’re seated at the table or after your food and drinks have arrived? Judge free zone.']
4 all orders! Use promo code: HBDPURPLE! https://t.co/IakM1pQOLb https://t.co/VaJpg2O6Cm']
5 , we are a free, fully open, mask free, people are working state. We have the best Governor with a big set that stands up to the DC lies. They fleeing CA so fast that Uhaul can't ke

In [97]:
twitter_drop = t_explode.drop_duplicates(subset=['tweet_slice'], keep='last')
twitter_drop

,county,original_tweet,tweets_list,tweet_str,tweet_slice
0,Texas test,[@tigerfan1975lsu @AmericanAir Thanks @America...,[@tigerfan1975lsu @AmericanAir Thanks @America...,['@tigerfan1975lsu @AmericanAir Thanks @Americ...,r ! Anyway we can get a badge or something to ...
0,Texas test,[@Heart10Looks3 @JosephChaplik I'm free to not...,[@tigerfan1975lsu @AmericanAir Thanks @America...,"[""@Heart10Looks3 @JosephChaplik I'm free to no...",ar a mask today. I exercise that right every d...
0,Texas test,[@johncardillo @AmericanAir @JohnKerry Don’t f...,[@tigerfan1975lsu @AmericanAir Thanks @America...,['@johncardillo @AmericanAir @JohnKerry Don’t ...,"et the fine of up to $35k, the @FAANews would ..."
0,Texas test,"[When dining in at restaurants, do y’all take ...",[@tigerfan1975lsu @AmericanAir Thanks @America...,"['When dining in at restaurants, do y’all take...",r mask(s) off once you’re seated at the table ...
0,Texas test,[💜TODAY ONLY🥳 Get a free Purple Mask Packette ...,[@tigerfan1975lsu @AmericanAir Thanks @America...,['💜TODAY ONLY🥳 Get a free Purple Mask Packette...,all orders! Use promo code: HBDPURPLE! https:/...
...,...,...,...,...,...
24,King County WA,[✨New Event!✨\nFree Drive-Thru Food Event!\nCo...,[✨New Event!✨\nFree Drive-Thru Food Event!\nCo...,['✨New Event!✨\nFree Drive-Thru Food Event!\nC...,to Marymoor Park at 6046 W Lk Sammamish Pkwy N...
24,King County WA,[@hzairs @itsjustkendyll you must be very naiv...,[✨New Event!✨\nFree Drive-Thru Food Event!\nCo...,['@hzairs @itsjustkendyll you must be very nai...,o not see how many ppl are voicing out about i...
24,King County WA,[✨New Event!✨\nFree Drive-Thru Food Event!\nCo...,[✨New Event!✨\nFree Drive-Thru Food Event!\nCo...,['✨New Event!✨\nFree Drive-Thru Food Event!\nC...,to Park Ridge Community Church at 3805 Maltby ...
24,King County WA,[Government announce new safety mearsures for ...,[✨New Event!✨\nFree Drive-Thru Food Event!\nCo...,['Government announce new safety mearsures for...,en and girls... a free Churchill face mask\n\n...


In [98]:
twitter_drop['tweet_str'].iloc[90]


'["@nbtm_sh @Shadow_TheWolf2 As I stated, it\'s FOSS. Free software can be literally modified however the user wants. There are mods out there that can mask the user agent and client name.\\n\\nSince Telegram is FOSS it is trivial to get the official client\'s unique key, even some Twitter clients do this actually"]'

In [99]:
twitter_drop['tweet_str'].iloc[88]


'["@ngoniofalltrade Honestly envying the fact that you\'re mask free and getting alot of fresh air out there. Mask yatopetsa kuno 😭"]'

## Cleans the tweets

In [100]:
df_clean = twitter_drop

In [101]:
# lower case
df_clean['tweet_text'] = df_clean['tweet_str'].str.lower()

# remove url links
df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))

# remove url/website that didn't use http, is only checking for .com websites 
# so words that are seperated by a . are not removed
df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))
# remove @mention
df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r'@mention', '', x))
# remove {link}
df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r'{link}', '', x))
# remove &text; html chars
df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r'&[a-z]+;', '', x))
# [video]
df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r"\[video\]", '', x))
# remove all remaining characters that aren't letters, white space, or 
# the following #:)(/\='] that are used in emojis or hashtags
df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', x))
df_clean

<ipython-input-101-89e914d04461>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['tweet_text'] = df_clean['tweet_str'].str.lower()
<ipython-input-101-89e914d04461>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))
<ipython-input-101-89e914d04461>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

,county,original_tweet,tweets_list,tweet_str,tweet_slice,tweet_text
0,Texas test,[@tigerfan1975lsu @AmericanAir Thanks @America...,[@tigerfan1975lsu @AmericanAir Thanks @America...,['@tigerfan1975lsu @AmericanAir Thanks @Americ...,r ! Anyway we can get a badge or something to ...,'tigerfanlsu americanair thanks americanair a...
0,Texas test,[@Heart10Looks3 @JosephChaplik I'm free to not...,[@tigerfan1975lsu @AmericanAir Thanks @America...,"[""@Heart10Looks3 @JosephChaplik I'm free to no...",ar a mask today. I exercise that right every d...,heartlooks josephchaplik i'm free to not wear ...
0,Texas test,[@johncardillo @AmericanAir @JohnKerry Don’t f...,[@tigerfan1975lsu @AmericanAir Thanks @America...,['@johncardillo @AmericanAir @JohnKerry Don’t ...,"et the fine of up to $35k, the @FAANews would ...",'johncardillo americanair johnkerry dont forge...
0,Texas test,"[When dining in at restaurants, do y’all take ...",[@tigerfan1975lsu @AmericanAir Thanks @America...,"['When dining in at restaurants, do y’all take...",r mask(s) off once you’re seated at the table ...,'when dining in at restaurants do yall take yo...
0,Texas test,[💜TODAY ONLY🥳 Get a free Purple Mask Packette ...,[@tigerfan1975lsu @AmericanAir Thanks @America...,['💜TODAY ONLY🥳 Get a free Purple Mask Packette...,all orders! Use promo code: HBDPURPLE! https:/...,'today only get a free purple mask packette on...
...,...,...,...,...,...,...
24,King County WA,[✨New Event!✨\nFree Drive-Thru Food Event!\nCo...,[✨New Event!✨\nFree Drive-Thru Food Event!\nCo...,['✨New Event!✨\nFree Drive-Thru Food Event!\nC...,to Marymoor Park at 6046 W Lk Sammamish Pkwy N...,'new event\nfree drive-thru food event\ncome t...
24,King County WA,[@hzairs @itsjustkendyll you must be very naiv...,[✨New Event!✨\nFree Drive-Thru Food Event!\nCo...,['@hzairs @itsjustkendyll you must be very nai...,o not see how many ppl are voicing out about i...,'hzairs itsjustkendyll you must be very naive ...
24,King County WA,[✨New Event!✨\nFree Drive-Thru Food Event!\nCo...,[✨New Event!✨\nFree Drive-Thru Food Event!\nCo...,['✨New Event!✨\nFree Drive-Thru Food Event!\nC...,to Park Ridge Community Church at 3805 Maltby ...,'new event\nfree drive-thru food event\ncome t...
24,King County WA,[Government announce new safety mearsures for ...,[✨New Event!✨\nFree Drive-Thru Food Event!\nCo...,['Government announce new safety mearsures for...,en and girls... a free Churchill face mask\n\n...,'government announce new safety mearsures for ...


## Do a spot check to make sure that the data has been cleaned well:

In [102]:
df_clean['tweet_text'].iloc[5]

"'if you live in supportive housing you are probably disabled enough to get the vaccine (or may be eligible in other ways) go out and get your vaccine so our mask-free non-distanced future can come sooner "

In [103]:
twitter_drop['tweet_str'].iloc[5]

"['If you live in supportive housing, you are probably disabled enough to get the vaccine (or may be eligible in other ways). Go out and get your vaccine so our mask-free non-distanced future can come sooner. https://t.co/zBsyD6VOvw']"

# Save as csv

In [104]:
df_clean.to_csv('cleaned3_tweets.csv')

#########################################################################################################